In [1]:
# Declaring path variables to make Basemap work in jupyter hub. Make sure that the conda environment path matches yours
import os
os.environ['PATH'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/bin:' + os.environ['PATH']
os.environ['PROJ_LIB'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/share/proj'

In [2]:
# General Tool Imports
import numpy as np
import pandas as pd
import collections

# Plotting Imports
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# Biota Imports
from biota.database import (get_samples,
                            get_sample_data)
from biota.util import (rarefy,
                        make_tree,
                        get_dms,
                        filter_dm_and_map,
                        get_tax_table)
import biota.plot as bpl # used for internal plotting code

# SourceTracker Imports
from ipyparallel import Client
from sourcetracker._sourcetracker import gibbs

# Clustering imports
from scipy.spatial.distance import pdist
from scipy.cluster import hierarchy

from skbio.stats.ordination import pcoa
from matplotlib.lines import Line2D

# Plot images in the notebook
%matplotlib inline

In [24]:
# query all southwestern data
query = {'Company': 'anadarko.petroleum.corporation','Well':{'$in':['silvertip.76.7.unit.a.6h']}}

#query = {'Company': 'anadarko.petroleum.corporation','Well': 'silvertip.76.7.unit.a.6h'}

samples, data, features = get_sample_data(query)

# Adding sequences per sample to the Samples dataframe
samples['SequencesPerSample'] = data.sum(axis=1) # sum the counts on a per row (per sample) basis

The columns ['WellType', 'MudType', 'Notes'] in the sample grouping dictionary were not included in the samples dataframe
The filtering function dropped 0 samples and 4698 features


In [14]:
# query all anadarko data
#query = {'Company': 'anadarko.petroleum.corporation'}
#samples, data, features = get_sample_data(query)

# Adding sequences per sample to the Samples dataframe
#samples['SequencesPerSample'] = data.sum(axis=1) # sum the counts on a per row (per sample) basis

The filtering function dropped 0 samples and 71695 features
The sample method dropped 59 samples with no sequence data.


In [18]:
data.shape

(103, 2103)

In [19]:
# A quick groupby summary to see what we've pulled down
samples.groupby(('SequenceSampleType', 'Well', 'CollectionDate')).size()

/home/jovyan/my-conda-envs/qiime2-2019.1/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  from ipykernel import kernelapp as app


SequenceSampleType  Well                      CollectionDate     
wellcutting         silvertip.76.7.unit.a.6h  2018-03-09 00:00:00     5
                                              2018-03-10 00:00:00     6
                                              2018-03-12 00:00:00     9
                                              2018-03-13 00:00:00     8
                                              2018-03-15 00:00:00    10
                                              2018-03-18 00:00:00     5
                                              2018-03-20 00:00:00     9
                                              2018-03-21 00:00:00     4
                                              3/3/2018               23
                                              3/4/2018               21
                                              3/5/2018                1
                                              3/7/2018                2
dtype: int64